In [157]:
import pandas as pd 
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
%matplotlib inline

In [158]:
#We aim to get four periods for accounting transactions
#We assign necessary dates to four period. Below, I fixed today's date and take "4 weeks periods" before today.

In [209]:
a1=d2-timedelta(days=27)
a2=d2-timedelta(days=21)
b1=d2-timedelta(days=20)
b2=d2-timedelta(days=14)
c1=d2-timedelta(days=13)
c2=d2-timedelta(days=7)
d1=d2-timedelta(days=6)
d2=pd.to_datetime("today")

In [210]:
#the ledger is loaded
dt = pd.read_excel('proje_github/ledger.xlsx')
dt.head()

,Date,No,Debit,Credit,Due Date,Remaining Amount
0,2018-12-31,003,3930.00,0.0,2018-12-31,0.00
1,2018-12-31,004,0.00,500.0,2018-12-31,0.00
2,2018-12-31,005,2754.15,0.0,2018-12-31,2754.15
3,2018-12-31,008,1809.16,0.0,2018-12-31,1809.16
4,2018-12-31,010,359.38,0.0,2018-12-31,0.00


In [211]:
#specific period names(p1,p2,p3,p4,p5) are added to the ledger table
dt4=dt

periods=[]
for i in dt4['Date']:
    if a1<=pd.to_datetime(i)<=a2:
       periods.append("p1")
    elif b1<=pd.to_datetime(i)<=b2:
       periods.append("p2")
    elif c1<=pd.to_datetime(i)<=c2:
       periods.append("p3")
    elif d1<=pd.to_datetime(i)<=d2:
       periods.append("p4")
    else:
        periods.append("p5")

dt4['Periods']=periods
dt4.sort_values(by=['Periods']).head()

,Date,No,Debit,Credit,Due Date,Remaining Amount,Periods
5631,2020-08-31,238,0.00,262.60,2020-08-31,0.00,p1
5633,2020-08-31,003,0.00,510.22,2020-08-31,0.00,p1
5634,2020-08-31,171,0.00,144.08,2020-08-31,0.00,p1
5660,2020-09-03,018,1245.35,0.00,2020-09-03,1245.35,p1
5644,2020-09-01,076,961.53,0.00,2020-09-01,961.53,p1


In [212]:
#the table is transofrmed into pivot_table in terms of specific period names
dt4=dt4.fillna(0)

dt4=dt4.pivot_table(columns=['Periods'],index=['No'],values=['Debit','Credit'], aggfunc='sum').fillna(0)

dt4.head()

Credit                          Debit                         
Periods      p1   p2   p3   p4        p5    p1   p2   p3   p4        p5
No                                                                     
001        0.00  0.0  0.0  0.0  23354.13   0.0  0.0  0.0  0.0  23354.13
002        0.00  0.0  0.0  0.0   2828.26   0.0  0.0  0.0  0.0   2828.26
003      510.22  0.0  0.0  0.0   6963.93   0.0  0.0  0.0  0.0   7757.62
004        0.00  0.0  0.0  0.0   1574.39   0.0  0.0  0.0  0.0   3012.16
005        0.00  0.0  0.0  0.0      0.00   0.0  0.0  0.0  0.0   2754.15

In [213]:
#the pivot_table above is arranged 
#credit is named as received payment and debit is named as sale
#The first Amount implies the net result of a customer 4 weeks before.

p1=pd.DataFrame([dt4.loc[:,'Debit'].loc[:,'p1'],dt4.loc[:,'Credit'].loc[:,'p1']])
p1=p1.T


p2=pd.DataFrame([dt4.loc[:,'Debit'].loc[:,'p2'],dt4.loc[:,'Credit'].loc[:,'p2']])
p2=p2.T

p3=pd.DataFrame([dt4.loc[:,'Debit'].loc[:,'p3'],dt4.loc[:,'Credit'].loc[:,'p3']])
p3=p3.T

p4=pd.DataFrame([dt4.loc[:,'Debit'].loc[:,'p4'],dt4.loc[:,'Credit'].loc[:,'p4']])
p4=p4.T

p5=pd.DataFrame([dt4.loc[:,'Debit'].loc[:,'p5'],dt4.loc[:,'Credit'].loc[:,'p5']])
p5=p5.T

result = pd.concat([p5,p1,p2,p3,p4], axis=1, sort=False).fillna(0)

result.columns=['a','b','c','d','e','f','g','h','i','j']
result['The First Amount']=result['a']-result['b']
result=result.drop("a", axis=1)
result=result.drop("b", axis=1)
result=result.fillna(0)

result = result[['The First Amount','c','d','e','f','g','h','i','j']]

result.columns=['The First Amount','Sale1','Received Payment1','Sale2','Received Payment2','Sale3','Received Payment3','Sale4','Received Payment4']

result.head()

,The First Amount,Sale1,Received Payment1,Sale2,Received Payment2,Sale3,Received Payment3,Sale4,Received Payment4
No,,,,,,,,,
001,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
002,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
003,793.69,0.0,510.22,0.0,0.0,0.0,0.0,0.0,0.0
004,1437.77,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
005,2754.15,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0


In [214]:
#We define totals for each period and arranged the table accordingly

result['Total1']=result['The First Amount']+result['Sale1']-result['Received Payment1']
result['Total2']=result['Total1']+result['Sale2']-result['Received Payment2']
result['Total3']=result['Total2']+result['Sale3']-result['Received Payment3']
result['Total4']=result['Total3']+result['Sale4']-result['Received Payment4']

result=result[['The First Amount','Sale1','Received Payment1','Total1','Sale2','Received Payment2','Total2','Sale3','Received Payment3','Total3','Sale4','Received Payment4','Total4']]
result.head()

,The First Amount,Sale1,Received Payment1,Total1,Sale2,Received Payment2,Total2,Sale3,Received Payment3,Total3,Sale4,Received Payment4,Total4
No,,,,,,,,,,,,,
001,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00
002,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00
003,793.69,0.0,510.22,283.47,0.0,0.0,283.47,0.0,0.0,283.47,0.0,0.0,283.47
004,1437.77,0.0,0.00,1437.77,0.0,0.0,1437.77,0.0,0.0,1437.77,0.0,0.0,1437.77
005,2754.15,0.0,0.00,2754.15,0.0,0.0,2754.15,0.0,0.0,2754.15,0.0,0.0,2754.15


In [215]:
#we have code table shoiwng the lists of of customers which we want to exclude from this report
#in code table, the ones with the code "REMOVE" will be excluded from the report
#firstly, theses codes are added to customer table.



customers = pd.read_excel('proje_github/customers.xlsx') 
customers=pd.merge(customers, code, how='left', on='No')
customers=customers.drop('Name_y',axis=1)
customers=customers.rename(columns={'Name_x':'Name'})
customers=customers.fillna(0)
customers.head()#BU TABLOYA ilgili sütunlar number olarak kaydedilsin. csv utf8

,No,Name,Amount,Supplier Amount,Code
0,001,Customer 1,0.00,0.0,0
1,002,Customer 2,0.00,0.0,0
2,003,Customer 3,283.47,0.0,0
3,004,Customer 4,1564.86,0.0,0
4,005,Customer 5,2754.15,0.0,0


In [216]:
#Then, the customer names with relevant codes are addded to the table above.
tablo=pd.merge(result,customers,how='left',on='No')
tablo=tablo.drop('Amount',axis=1)
tablo=tablo[['No','Name', 'The First Amount', 'Sale1', 'Received Payment1', 'Total1',
       'Sale2', 'Received Payment2', 'Total2', 'Sale3', 'Received Payment3',
       'Total3', 'Sale4', 'Received Payment4', 'Total4', 
       'Supplier Amount', 'Code']]
tablo.head()

,No,Name,The First Amount,Sale1,Received Payment1,Total1,Sale2,Received Payment2,Total2,Sale3,Received Payment3,Total3,Sale4,Received Payment4,Total4,Supplier Amount,Code
0,001,Customer 1,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0
1,001,Customer 246,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0
2,002,Customer 2,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0
3,002,Customer 247,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0
4,003,Customer 3,793.69,0.0,510.22,283.47,0.0,0.0,283.47,0.0,0.0,283.47,0.0,0.0,283.47,0.0,0


In [217]:
#customers with "REMOVE" codes are excluded
tablo=tablo[tablo['Code']!="REMOVE"]
tablo=tablo.fillna(0)
tablo.head()

C:\Users\huseyin\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,No,Name,The First Amount,Sale1,Received Payment1,Total1,Sale2,Received Payment2,Total2,Sale3,Received Payment3,Total3,Sale4,Received Payment4,Total4,Supplier Amount,Code
0,001,Customer 1,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0
1,001,Customer 246,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0
2,002,Customer 2,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0
3,002,Customer 247,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0
4,003,Customer 3,793.69,0.0,510.22,283.47,0.0,0.0,283.47,0.0,0.0,283.47,0.0,0.0,283.47,0.0,0


In [218]:
#some customers are also our supplier so if our company is indebted to the customer, we can see this amount in supplier amount.
tablo['Net Amount']=tablo['Total4']+tablo['Supplier Amount']

#+ weeks Total Sale & Received payment are added to the list.
tablo['Last 4 Weeks Sale']=tablo['Sale1']+tablo['Sale2']+tablo['Sale3']+tablo['Sale4']
tablo['Last 4 Weeks Received Payments']=tablo['Received Payment1']+tablo['Received Payment2']+tablo['Received Payment3']+tablo['Received Payment4']
tablo.head()

,No,Name,The First Amount,Sale1,Received Payment1,Total1,Sale2,Received Payment2,Total2,Sale3,Received Payment3,Total3,Sale4,Received Payment4,Total4,Supplier Amount,Code,Net Amount,Last 4 Weeks Sale,Last 4 Weeks Received Payments
0,001,Customer 1,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0,0.00,0.0,0.00
1,001,Customer 246,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0,0.00,0.0,0.00
2,002,Customer 2,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0,0.00,0.0,0.00
3,002,Customer 247,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0,0.00,0.0,0.00
4,003,Customer 3,793.69,0.0,510.22,283.47,0.0,0.0,283.47,0.0,0.0,283.47,0.0,0.0,283.47,0.0,0,283.47,0.0,510.22


In [219]:
tablo=tablo[tablo['Net Amount']>1]
#the items with less than and equal to 1 are excluded
tablo.head()

,No,Name,The First Amount,Sale1,Received Payment1,Total1,Sale2,Received Payment2,Total2,Sale3,Received Payment3,Total3,Sale4,Received Payment4,Total4,Supplier Amount,Code,Net Amount,Last 4 Weeks Sale,Last 4 Weeks Received Payments
4,003,Customer 3,793.69,0.0,510.22,283.47,0.0,0.0,283.47,0.0,0.0,283.47,0.0,0.0,283.47,0.0,0,283.47,0.0,510.22
5,003,Customer 248,793.69,0.0,510.22,283.47,0.0,0.0,283.47,0.0,0.0,283.47,0.0,0.0,283.47,0.0,0,283.47,0.0,510.22
6,004,Customer 4,1437.77,0.0,0.00,1437.77,0.0,0.0,1437.77,0.0,0.0,1437.77,0.0,0.0,1437.77,0.0,0,1437.77,0.0,0.00
7,004,Customer 249,1437.77,0.0,0.00,1437.77,0.0,0.0,1437.77,0.0,0.0,1437.77,0.0,0.0,1437.77,0.0,0,1437.77,0.0,0.00
8,005,Customer 5,2754.15,0.0,0.00,2754.15,0.0,0.0,2754.15,0.0,0.0,2754.15,0.0,0.0,2754.15,0.0,0,2754.15,0.0,0.00


In [220]:
# in ledger table, we have remaining amounts indicating the remaining amonut after a payment is received from the customer.
#we can monitor the remaining amount from a specific invoice(sales)
#Here, I arranged a list of 60 days overdue in customer debt
#fthe overdue transactions in the ledger table are determined.


from datetime import timedelta
Term=[]
for i in dt['Date']:
    if pd.Timestamp(i)+ timedelta(days=60)< pd.to_datetime("today"):
        Term.append("yes")
    else:
        Term.append("no")
dt['Term']=Term
dt=dt[dt['Term']=="yes"]
d=dt.groupby('No')['Remaining Amount'].sum()
overdue=pd.DataFrame(d)

tablo=pd.merge(tablo,overdue,how='left',on='No')
tablo=tablo.fillna(0)
son=[]
for i in range(len(tablo['Remaining Amount'])):
    if tablo['Remaining Amount'][i]<0:
        son.append(0)
    elif tablo['Remaining Amount'][i]<tablo['Net Amount'][i]:
        son.append(tablo['Remaining Amount'][i])
    else:
        son.append(tablo['Net Amount'][i])
        
tablo['Overdue Amount']=son



tablo.head()


,No,Name,The First Amount,Sale1,Received Payment1,Total1,Sale2,Received Payment2,Total2,Sale3,...,Sale4,Received Payment4,Total4,Supplier Amount,Code,Net Amount,Last 4 Weeks Sale,Last 4 Weeks Received Payments,Remaining Amount,Overdue Amount
0,003,Customer 3,793.69,0.0,510.22,283.47,0.0,0.0,283.47,0.0,...,0.0,0.0,283.47,0.0,0,283.47,0.0,510.22,283.47,283.47
1,003,Customer 248,793.69,0.0,510.22,283.47,0.0,0.0,283.47,0.0,...,0.0,0.0,283.47,0.0,0,283.47,0.0,510.22,283.47,283.47
2,004,Customer 4,1437.77,0.0,0.00,1437.77,0.0,0.0,1437.77,0.0,...,0.0,0.0,1437.77,0.0,0,1437.77,0.0,0.00,-127.09,0.00
3,004,Customer 249,1437.77,0.0,0.00,1437.77,0.0,0.0,1437.77,0.0,...,0.0,0.0,1437.77,0.0,0,1437.77,0.0,0.00,-127.09,0.00
4,005,Customer 5,2754.15,0.0,0.00,2754.15,0.0,0.0,2754.15,0.0,...,0.0,0.0,2754.15,0.0,0,2754.15,0.0,0.00,2754.15,2754.15
